<a href="https://colab.research.google.com/github/Kim-yongbeom/machine-learning/blob/main/data21/%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_surprise%EC%9D%B4%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

In [2]:
import surprise

In [3]:
print(surprise.__version__)

1.1.1


In [4]:
from surprise import SVD ## 고유값분해(행렬을 더 낮은 차원으로 분해)해서 다시 원본으로 살리면서 잠재적인 값을 추론하는 방법
from surprise import Dataset, Reader ## SVD를 사용하기 위한 데이터셋 만들어주는 클래스
                                     ## 사용자, 아이템, 평점
from surprise import accuracy ## RMSE, MAE, cross-validation(k-fold)
from surprise.model_selection import train_test_split ## 훈련/검증 데이터 분류

In [5]:
## 1. 데이터셋을 만들어주자(사용자, 아이템, 평점), 훈련/검증 데이터 분류
data = Dataset.load_builtin('ml-100k')
data

In [6]:
trainset, testset = train_test_split(data, test_size=0.25, random_state= 0)

In [7]:
trainset

In [8]:
testset[:5]

[('120', '282', 4.0),
 ('882', '291', 4.0),
 ('535', '507', 5.0),
 ('697', '244', 5.0),
 ('751', '385', 4.0)]

In [9]:
## 2. SVD() 모델 선정
algo = SVD()

In [10]:
## 3. 훈련용 데이터로 fit()
algo.fit(trainset)

In [11]:
## 4. 검증용 데이터로 예측
algo.test(testset)[:5]

[Prediction(uid='120', iid='282', r_ui=4.0, est=3.4254296375502005, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.3155833630481215, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.051516398603681, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.611840980015436, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.3053657193470705, details={'was_impossible': False})]

In [12]:
## 5. 정확도 계산
predictions = algo.test(testset)[:5]
predictions

[Prediction(uid='120', iid='282', r_ui=4.0, est=3.4254296375502005, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.3155833630481215, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.051516398603681, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.611840980015436, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.3053657193470705, details={'was_impossible': False})]

In [13]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions]

[('120', '282', 3.4254296375502005),
 ('882', '291', 3.3155833630481215),
 ('535', '507', 4.051516398603681),
 ('697', '244', 3.611840980015436),
 ('751', '385', 3.3053657193470705)]

In [14]:
import pandas as pd

In [15]:
## csv를 SVD하기 위한 DataSet를 주기 위해서는
## 1) df <---- csv, 2) DataSet <---- df

In [16]:
ratings = pd.read_csv('ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [17]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [18]:
# min = 0.5 , max = 5
reader = Reader(rating_scale=(0.5, 5.0))

In [19]:
## SVD에서 사용할 수 있는 데이터 셋으로 만들어주자
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
data

In [20]:
trainset, testset = train_test_split(data, test_size=0.25, random_state= 0)

In [21]:
from surprise.dataset import DatasetAutoFolds
reader = Reader(line_format='user item rating timestamp',sep=',',rating_scale=(0.5,5))
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv',reader=reader)
trainset=data_folds.build_full_trainset()

In [35]:
algo = SVD()
algo.fit(trainset)

In [36]:
algo.test(testset)[:5]

[Prediction(uid=63, iid=2000, r_ui=3.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=31, iid=788, r_ui=2.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=159, iid=6373, r_ui=4.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=105, iid=81564, r_ui=3.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=394, iid=480, r_ui=3.0, est=3.501556983616962, details={'was_impossible': False})]

In [23]:
predictions = algo.test(testset)
predictions

[Prediction(uid=63, iid=2000, r_ui=3.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=31, iid=788, r_ui=2.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=159, iid=6373, r_ui=4.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=105, iid=81564, r_ui=3.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=394, iid=480, r_ui=3.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=181, iid=587, r_ui=5.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=224, iid=3072, r_ui=5.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=328, iid=1391, r_ui=4.5, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=50, iid=104283, r_ui=3.5, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=125, iid=176371, r_ui=3.5, est=3.501556983616962, details={'was_impossible': False}),
 

In [24]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions]

[(63, 2000, 3.501556983616962),
 (31, 788, 3.501556983616962),
 (159, 6373, 3.501556983616962),
 (105, 81564, 3.501556983616962),
 (394, 480, 3.501556983616962),
 (181, 587, 3.501556983616962),
 (224, 3072, 3.501556983616962),
 (328, 1391, 3.501556983616962),
 (50, 104283, 3.501556983616962),
 (125, 176371, 3.501556983616962),
 (372, 709, 3.501556983616962),
 (448, 2951, 3.501556983616962),
 (438, 6617, 3.501556983616962),
 (169, 3, 3.501556983616962),
 (111, 96432, 3.501556983616962),
 (328, 34, 3.501556983616962),
 (117, 595, 3.501556983616962),
 (260, 7022, 3.501556983616962),
 (521, 32, 3.501556983616962),
 (42, 4482, 3.501556983616962),
 (545, 1367, 3.501556983616962),
 (232, 43904, 3.501556983616962),
 (179, 648, 3.501556983616962),
 (387, 8972, 3.501556983616962),
 (288, 2372, 3.501556983616962),
 (604, 62, 3.501556983616962),
 (160, 1748, 3.501556983616962),
 (482, 1961, 3.501556983616962),
 (82, 4386, 3.501556983616962),
 (102, 349, 3.501556983616962),
 (568, 3363, 3.501556983

In [25]:
accuracy.rmse(predictions)

RMSE: 1.0388


1.0387802469854106

In [26]:
from surprise.model_selection import cross_validate

In [27]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8744  0.8742  0.8771  0.8733  0.8703  0.8739  0.0022  
MAE (testset)     0.6700  0.6685  0.6752  0.6741  0.6710  0.6718  0.0025  
Fit time          6.25    5.00    4.98    4.92    5.00    5.23    0.51    
Test time         0.16    0.28    0.15    0.17    0.28    0.21    0.06    


{'fit_time': (6.245432376861572,
  4.996145248413086,
  4.981090545654297,
  4.924224376678467,
  5.0027570724487305),
 'test_mae': array([0.66995076, 0.66851679, 0.67524888, 0.67409295, 0.67096598]),
 'test_rmse': array([0.87440447, 0.87420403, 0.87708442, 0.87328942, 0.8703036 ]),
 'test_time': (0.15828418731689453,
  0.28218865394592285,
  0.15303969383239746,
  0.1697249412536621,
  0.279965877532959)}

In [28]:
import time
start = time.time()
start

1640239543.8499742

In [29]:
end = time.time()
end

1640239543.8600795

In [30]:
end-start

0.010105371475219727

In [31]:
movies = pd.read_csv('/content/movies (1).csv')
movieIds = ratings[ratings['userId'] == 9]['movieId']
if movieIds[movieIds==42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [32]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [38]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Shawshank Redemption, The (1994) : 4.23430773333891
Casablanca (1942) : 4.152284350925699
Lawrence of Arabia (1962) : 4.14170252592804
High Noon (1952) : 4.126803374516306
Fight Club (1999) : 4.112935497995027
Outlaw Josey Wales, The (1976) : 4.109039035161956
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001) : 4.108701858843193
Nausicaä of the Valley of the Wind (Kaze no tani no Naushika) (1984) : 4.085217854026251
Lord of the Rings: The Return of the King, The (2003) : 4.080926352530309
Departed, The (2006) : 4.058079405126434


In [37]:
top_predictions = recomm_movie_by_surprise(algo,9,unseen_movies)
top_predictions

[(1204, 'Shawshank Redemption, The (1994)', 4.23430773333891),
 (1283, 'Casablanca (1942)', 4.152284350925699),
 (7153, 'Lawrence of Arabia (1962)', 4.14170252592804),
 (3508, 'High Noon (1952)', 4.126803374516306),
 (48516, 'Fight Club (1999)', 4.112935497995027),
 (318, 'Outlaw Josey Wales, The (1976)', 4.109039035161956),
 (912,
  "Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",
  4.108701858843193),
 (2959,
  'Nausicaä of the Valley of the Wind (Kaze no tani no Naushika) (1984)',
  4.085217854026251),
 (4973,
  'Lord of the Rings: The Return of the King, The (2003)',
  4.080926352530309),
 (7099, 'Departed, The (2006)', 4.058079405126434)]